In [ ]:
# Imports necessary Packages
import numpy as np
import scipy.stats as sps
import pandas as pd

# Specific Plotting Packages
import matplotlib.pyplot as plt
from matplotlib import cm
from matplotlib import patches
from mpl_toolkits.mplot3d import Axes3D

# sklearn
from sklearn.preprocessing import normalize

# Import Custom Packages
import supplemental_funcs as sf
import example_master as EM # sets values for example

# to show in notebook
%matplotlib inline

## Table of Contents
* Bayesian Solution to Fixed Plate Problem
* Bayesian Solution to Wobbly Plate Problem

In [ ]:
# set a random seed
# np.random.seed(24513)

In [ ]:
plt.rcParams.update({'font.size': 14})

In [ ]:
# defines a general color pallette
col_highlight = 'xkcd:yellow'
col_blue = 'xkcd:sky'
col_red = 'xkcd:red orange'

In [ ]:
# save all figures dictionary
# fig_save_dictionary { 'filename' : figname }
fig_all_master = {}

# Stationary or Wobbly Plate?

Defines model map $Q:\Lambda\rightarrow\mathcal{D}$ as:

\begin{align}
Q(\lambda)=\begin{pmatrix}0.6 & 0.7 \\
0.8 & 0.6
\end{pmatrix}\cdot \begin{pmatrix} \lambda_1 \\ \lambda_2 \end{pmatrix} +3
\end{align}

where $\vec{x}_A=(0.6,0.7)$ and $\vec{x}_B=(0.8,0.6)$ are the two locations and $y_0=3$ is the height of the plate above the origin.

In [ ]:
print('Linear Map X = ')
print(EM.locX_mat)
print()
print('y0 = ',EM.y0)
print()

In [ ]:
# Use parameter file EM to define appropriate parameters
n_obs = EM.n_obs # number of observations
locations = EM.locX_mat # location of observations
y0 = EM.y0

# mean of parameter slope or true fixed value for the slope
lam_mean = EM.lam_dagger

# define the observed parameters
obs_mean = EM.obs_mean
obs_cov = EM.obs_cov
obs_dist = sps.multivariate_normal(obs_mean,obs_cov)

# obtain inverse lambda target distribution
loc_inverse = EM.locX_mat_inv
lam_cov = EM.lam_gen_cov
lam_target_dist = sps.multivariate_normal(lam_mean,lam_cov)

# observed sample
q_obs = obs_dist.rvs(n_obs)

In [ ]:
print(obs_mean)
print(obs_cov)
print(lam_mean)

### Plot Different Scenario Solutions

Here I plot the data space and the corresponding inverse solutions in the parameter space for the two different kinds of problems:

1. Parameter **Estimation** Problem
2. Parameter **Distribution** Problem

These correspond to a situation where the plate has a fixed slope (stationary plate) or has a random slope (wobbly plate), respectively.

In [ ]:
# get eigenvalues and eigenvectors of matrix
eig_val, eig_vec = np.linalg.eig(obs_cov)

In [ ]:
# make lam-sapce grid
lamx = np.linspace(-8, 8, 100)
lamy = np.linspace(-8, 8, 100)
lamX, lamY = np.meshgrid(lamx, lamy)
eval_lamXY = np.vstack([lamX.ravel(),lamY.ravel()])


# make data-space grid
qx = np.linspace(0, 6, 100)
qy = np.linspace(0, 6, 100)
qX, qY = np.meshgrid(qx, qy)
eval_qXY = np.vstack([qX.ravel(),qY.ravel()])

# eval points and reshape
Z1 = lam_target_dist.pdf(eval_lamXY.T).reshape(lamX.shape)
Z2 = obs_dist.pdf(eval_qXY.T).reshape(qX.shape)

# parameter estimation: sampling distribution
this_n = 10
lam_hat_dist = sps.multivariate_normal(lam_mean,1/this_n*lam_cov)
Z_hat = lam_hat_dist.pdf(eval_lamXY.T).reshape(lamX.shape)

# decompose the eigenvalue matrix of lam
exlam_eig, exlam_eigV =  np.linalg.eig(lam_cov)

In [ ]:
# for making the ellipses easier
def make_cov_ellipse(center,cov,area=0.9,ellipse_args={}):
    # sets an area for the circles that includes area% of data
    this_area = sps.chi2.ppf(area,df=2) 
    this_center = center
    
    # eigen decomp covariance for ellipse parameters
    this_eig_decomp = np.linalg.eig(cov)
    
    # eigen value widths and heights
    this_eig0, this_eig1 = this_eig_decomp[0]
    this_width = 2*np.sqrt(this_area/2*this_eig0)
    this_height = 2*np.sqrt(this_area/2*this_eig1)
    
    # rotation
    this_eig_vec = this_eig_decomp[1][:,0]
    rot_angle = np.arctan(this_eig_vec[1]/this_eig_vec[0])*180/np.pi

    this_ellipse = patches.Ellipse(xy=this_center,angle=rot_angle,
                               width=this_width,height=this_height,
                               **ellipse_args)
    return this_ellipse

In [ ]:
fig_intro_solutions, axes = plt.subplots(2,2)
fig_intro_solutions.tight_layout()
fig_intro_solutions.set_figwidth(8)
fig_intro_solutions.set_figheight(8)

these_ellipse_args = {'fc':'None','edgecolor':'C1','alpha':0.7}
this_cycler = plt.rcParams['axes.prop_cycle']
colors = this_cycler.by_key()['color']
# create 4 figure
for i,ax_row in enumerate(axes):
    for j,ax in enumerate(ax_row):
        # lambda space
        if j==0:
            if i==1:
                contour = ax.contour(lamX,lamY,Z1)
                ax.set_title('Parameter Distribution Solution')
                ax.plot(lam_mean[0],lam_mean[1],'or',label='$\mu_{\lambda}$')
            else:
                for k,n in enumerate([1,5,20]):
                    these_ellipse_args['edgecolor'] = colors[k+1]
                    this_ellipse = make_cov_ellipse(lam_mean,1/n*lam_cov,area=0.99,
                                                    ellipse_args=these_ellipse_args)
                    ax.add_artist(this_ellipse)
                    
                    # label curves
                    exlam_eig
                    this_loc = lam_mean+0.6*1/n*exlam_eig[1]*exlam_eigV[:,1]
                    adjusts = [np.array([0,0]),np.array([0,-1.5]),np.array([0.5,0.6])]
                    halign = 'right' if k==0 else 'left'
                    ax.annotate('$n={}$'.format(n),xy=this_loc+adjusts[k],
                               color=colors[k+1],va='top',ha=halign)
                    
                ax.set_xlim(lamx[0],lamx[-1])
                ax.set_ylim(lamy[0],lamy[-1])
                ax.set_title('Parameter Estimation Solution')
                ax.plot(lam_mean[0],lam_mean[1],'or',label='$\\hat{\lambda}$')
            ax.set_xlabel('$\lambda_1$')
            ax.set_ylabel('$\lambda_2$')
            
#             contour.collections[-4].set_label('Random Slope')
            ax.legend()
            
        else:
            ax.contour(qX,qY,Z2)
#             ax_dat.annotate('',xy=obs_mean-2.5*eig_val[1]*eig_vec[:,1],xytext=obs_mean,
#                             arrowprops=dict(arrowstyle='->'))

#             ax_dat.annotate('',xy=obs_mean-2.5*eig_val[0]*eig_vec[:,0],xytext=obs_mean,
#                             arrowprops=dict(arrowstyle='->'))
            labels = ['$\\bar{q}$','$\mu_q$']
            ax.plot(obs_mean[0],obs_mean[1],'or',label=labels[i])
            ax.set_title('Distribution of Heights')
            ax.set_xlabel('$y_A$')
            ax.set_ylabel('$y_B$')
            ax.legend(loc='lower left')

# figure name will be 
this_fig_name = 'fig_intro_problem_types.png'
fig_all_master[this_fig_name] = fig_intro_solutions

# Save just this fig
# fig_intro_solutions.savefig('../'+this_fig_name)

# Bayesian Solution to Parameter Estimation Problem

We have the likelihood of $q$ given $\vec{\lambda}$ as multivariate normal with observed mean $\mu_q$ and known covariance $\Sigma_q$.

The conjugate prior will be a normal distribution. Thus our Bayesian model will be:

\begin{align*}
\vec{\lambda} &\sim \text{mv_normal}(\mu_0,\Sigma_0) \\
Q(\vec{\lambda}) &= X\vec{\lambda} + y_0 \quad,\quad \text{where } X=\begin{pmatrix}0.6 & 0.7 \\
0.8 & 0.6
\end{pmatrix} \\
q \mid \vec{\lambda} &\sim \text{mv_normal}(Q(\vec{\lambda}),\Sigma_q)
\end{align*}

Given this model, we can derive the exact posterior distribution for $n$ observations $q_i$ of $q$:

\begin{align*}
Q(\vec{\lambda}) \mid q_i &\sim \text{mv_normal}(\mu_{Qpost},\Sigma_{Qpost}) \\
\Sigma_{Qpost} &= \left( (X\Sigma_0 X^T)^{-1} + n\Sigma_q^{-1}\right)^{-1} \\
\mu_{Qpost} &= \Sigma_{Qpost} \cdot\left( (X\Sigma_0 X^T)^{-1} (X\mu_0+y_0) + n \Sigma_q^{-1}\mu_{q_i} \right) \\
\end{align*}

where $\mu_{q_i}$ is the mean of the observations $q_i$.

Thus, the exact posterior distribution of $\vec{\lambda}$ will be:

\begin{align}
\vec{\lambda} \mid q_i &\sim \text{mv_normal}(\mu_{post},\Sigma_{post}) \\
\\
\Sigma_{post} &= (X^{-1})\Sigma_{Qpost}(X^{-1})^T \\
& = (X^{-1})\left[ (X\Sigma_0 X^T)^{-1} + n\Sigma_q^{-1}\right]^{-1}(X^{-1})^T \\
&= (X)^{-1}\left[ (X\Sigma_0 X^T)^{-1} + n\Sigma_q^{-1}\right]^{-1}(X^{T})^{-1} \\
&=\left[X^{T} \left( (X\Sigma_0 X^T)^{-1} + n\Sigma_q^{-1} \right) X \right]^{-1} \\
&=\left[X^{T}(X^{T})^{-1}\Sigma_0^{-1} X^{-1}X + n X^T \Sigma_q^{-1}X \right]^{-1}
\\
&=\left[\Sigma_0^{-1} + n X^T \Sigma_q^{-1}X \right]^{-1}\\
\\
\mu_{post} &= X^{-1}(\mu_{Qpost}-y_0)\\
&= X^{-1}\left[\Sigma_{Qpost} \cdot\left( (X\Sigma_0 X^T)^{-1} (X\mu_0+y_0) + n \Sigma_q^{-1}\mu_{q_i} \right)-y_0\right] \\
&= X^{-1}\left[X X^{-1}\cdot\Sigma_{Qpost}\cdot (X^T)^{-1}X^T \cdot\left( (X\Sigma_0 X^T)^{-1} (X\mu_0+y_0) + n \Sigma_q^{-1}\mu_{q_i} \right)-y_0\right] \\
&= X^{-1}\left[X \Sigma_{post} X^T \cdot\left( (X\Sigma_0 X^T)^{-1} (X\mu_0+y_0) + n \Sigma_q^{-1}\mu_{q_i} \right)-y_0\right] \\
&= \left[ \Sigma_{post} \cdot\left( \Sigma_0^{-1}\mu_0+ \Sigma_0^{-1}X^{-1}  y_0 + X^T n \Sigma_q^{-1}\mu_{q_i} \right)-X^{-1}y_0\right] \\
&= \left[ \Sigma_{post} \cdot\left( \Sigma_0^{-1}\mu_0 + X^T n \Sigma_q^{-1}\mu_{q_i} \right) + \Sigma_{post}\Sigma_0^{-1}X^{-1}  y_0-X^{-1}y_0\right] \\
&= \left[ \Sigma_{post} \cdot\left( \Sigma_0^{-1}\mu_0 + X^T n \Sigma_q^{-1}\mu_{q_i} \right) +\Sigma_{post}\left(\Sigma_0^{-1}  -\Sigma_{post}^{-1}\right)X^{-1}y_0\right] \\
&= \left[ \Sigma_{post} \cdot\left( \Sigma_0^{-1}\mu_0 + X^T n \Sigma_q^{-1}\mu_{q_i} \right) +\Sigma_{post}\left(\Sigma_0^{-1}  -\Sigma_0^{-1} - X^T n\Sigma_q^{-1}X\right)X^{-1}y_0\right] \\
&= \Sigma_{post} \cdot\left( \Sigma_0^{-1}\mu_0 + n X^T \Sigma_q^{-1}(\mu_{q_i}-y_0)\right)  \\
\end{align}

This is confirmed in the book by [Kaipio 2005](http://skyline.ucdenver.edu/record=b2382548~S0).

### Quantitative Illustration

In [ ]:
# propose prior
sig0 = EM.sigma0_sphere
prior_mean = np.array([0,0]) # centered around the plate's slope being flat
prior_cov = sig0*np.eye(2)  # value chosen to be very broad over the space
prior_prec = np.linalg.inv(prior_cov)
prior_dist = sps.multivariate_normal(prior_mean,prior_cov)

In [ ]:
# for plotting
prior_sample = prior_dist.rvs(500)
Z_prior = prior_dist.pdf(eval_lamXY.T).reshape(lamX.shape)

In [ ]:
# plot prior
plt.plot(lam_mean[0],lam_mean[1],'or',label='Fixed Slope')
plt.scatter(prior_sample.T[0],prior_sample.T[1],label='prior sample',alpha=0.5)
plt.contour(lamX,lamY,Z_prior)
plt.legend()

> Just want to check the relationship between the initial distribution and fixed slope value. The fixed slope value $\lambda^\dagger$ is within the support of the initial distribution, which is quite broad by assumption.

**Check that posterior equations are correct**

Here I just validate the derivation equations above computationally.

In [ ]:
# define theoretical posterior distribution

# get observed mean and precision
# obs_sample_mean = q_obs.mean(axis=0)
obs_sample_mean = obs_mean
obs_prec = np.linalg.inv(obs_cov)

# get posterior covariance
post_lam_prec = prior_prec+np.linalg.multi_dot([locations.T,n_obs*obs_prec,locations])
post_lam_cov = np.linalg.inv(post_lam_prec)

# get push-forward distribution parameters
q_prior_mean = np.dot(locations,prior_mean)+y0 ##CHANGE TO Q(mean)
q_prior_cov = np.linalg.multi_dot([locations,prior_cov,locations.T])
q_prior_prec = np.linalg.inv(q_prior_cov)

# get weighted sum for posterior q
weighted_mean = np.dot(q_prior_prec,q_prior_mean) + np.dot(n_obs*obs_prec,obs_sample_mean)
post_q_cov = np.linalg.multi_dot([locations,post_lam_cov,locations.T])
post_q_mean = np.dot(post_q_cov,weighted_mean)

# inverse transform to obtain lamda mean
post_lam_mean = np.dot(loc_inverse, post_q_mean-y0)

# simplified form of posterior mean
val = np.linalg.multi_dot([prior_prec,prior_mean])+np.linalg.multi_dot([locations.T,n_obs*obs_prec,obs_sample_mean-y0])
post_lam_mean_TEST = np.dot(post_lam_cov,val)

# posterior distribution
post_dist = sps.multivariate_normal(post_lam_mean,post_lam_cov)

In [ ]:
print(post_lam_mean,post_lam_mean_TEST)

> The following plot is just me checking that the method of finding the posterior mean for $q$ and then inverting to obtain the posterior mean for $\lambda$ is a reasonable appraoch.

In [ ]:
# observed sample
plt.contour(qX,qY,Z2)
plt.scatter(q_obs.T[0],q_obs.T[1])
plt.title('Observed Sample of $n={}$'.format(n_obs))
plt.xlabel('$q_A$')
plt.ylabel('$q_B$')
plt.plot(post_q_mean[0],post_q_mean[1],'or',label='posterior mean for $q$')
plt.legend()

## Obtain the Posterior for Different Size Samples

The goal of this next portion is to obtain the posterior parameters for different sample sizes.

**Obtain the Posterior Parameters**

The following function is defined to get the exact posterior parameters using the initial parameters and the observed parameters.

In [ ]:
this_info = {'prior_mean': prior_mean,
             'prior_prec': prior_prec,
             'prior_cov': prior_cov,
             'linear_map': locations,
             'y0': y0,
             'obs_prec': obs_prec
            }

def get_example_posterior_param(sample, info=this_info):
    '''
    Gets the posterior mean and covariance for this specific example.
    prior_info: dictionary of prior_mean, prior_cov, prior_prec, 
                            linear_map, y0, and known obs_prec
    Sample: should be number of samples n x 2x2
    returns: posterior mean and posterior covariance
    '''
    # get info params
    this_prior_prec = info['prior_prec']
    this_prior_cov = info['prior_cov']
    this_prior_mean = info['prior_mean']
    
    X = info['linear_map']
    b = info['y0']
    this_obs_prec = info['obs_prec']
    
    # get sample information
    n = sample.shape[0]
    sample_mean = sample.mean(axis=0)
    
    
    # get posterior covariance
    this_post_prec = this_prior_prec + np.linalg.multi_dot([X.T,n*this_obs_prec,X])
    this_post_cov = np.linalg.inv(this_post_prec)

    
    # get posterior mean
    bias_move = np.linalg.multi_dot([this_prior_prec,this_prior_mean]) 
    data_move = np.linalg.multi_dot([X.T,n*this_obs_prec,sample_mean-b])
    this_post_mean = np.dot(this_post_cov,(bias_move+data_move))
    
    return this_post_mean, this_post_cov

The following plot shows how the Bayesian posterior with $n$ samples converges to a fixed point estimate using the likelihood.

In [ ]:
# for plotting
Z_post = post_dist.pdf(eval_lamXY.T).reshape(lamX.shape)

In [ ]:
# plot posterior
plt.plot(lam_mean[0],lam_mean[1],'or',label='Fixed Slope')
plt.scatter(prior_sample.T[0],prior_sample.T[1],label='prior sample',alpha=0.5)
plt.contour(lamX,lamY,Z_post)
# plt.xlim(0,5)
# plt.ylim(0,5)
plt.legend()

### Figure of Four for Convergence Plot

In [ ]:
# make lam-sapce grid
lamx_small = np.linspace(0, 2, 100)
lamy_small = np.linspace(0, 2, 100)
lamX_small, lamY_small = np.meshgrid(lamx_small, lamy_small)
eval_lamXY_small = np.vstack([lamX_small.ravel(),lamY_small.ravel()])

In [ ]:
test = post_dist.rvs(100).T
plt.scatter(test[0],test[1])
plt.contour(lamX_small,lamY_small,
            post_dist.pdf(eval_lamXY_small.T).reshape(lamX_small.shape))

In [ ]:
q_obs[0:5]

In [ ]:
get_example_posterior_param(q_obs[::100])

In [ ]:
fig_param_converge, axes = plt.subplots(2,2)
fig_param_converge.set_figheight(8)
fig_param_converge.set_figwidth(8)
fig_param_converge.tight_layout(h_pad=2)

axes = axes.flatten()
n_size = [0,1,100,250,q_obs.shape[0]]
for i, ax in enumerate(axes):
    ax.plot(lam_mean[0],lam_mean[1],'or',label='Fixed Slope')
    if i == 0:
        Z_prior = prior_dist.pdf(eval_lamXY.T).reshape(lamX.shape)
        ax.contour(lamX,lamY,Z_prior)
        ax.set_title('Prior')
    elif i==1:
        this_params = get_example_posterior_param(q_obs[0:n_size[i]])
        this_post_pdf = sps.multivariate_normal(this_params[0],this_params[1])
        Z_post = this_post_pdf.pdf(eval_lamXY.T).reshape(lamX.shape)
        ax.contour(lamX,lamY,Z_post)
        ax.set_title('Posterior $n={}$'.format(n_size[i]))
    else:
        this_params = get_example_posterior_param(q_obs[0:n_size[i]])
        this_post_pdf = sps.multivariate_normal(this_params[0],this_params[1])
        Z_post = this_post_pdf.pdf(eval_lamXY_small.T).reshape(lamX_small.shape)
        ax.contour(lamX_small,lamY_small,Z_post)
        ax.set_title('Posterior $n={}$'.format(n_size[i]))

# figure name will be 
this_fig_name = 'bayes_param_converge.png'
fig_all_master[this_fig_name] = fig_param_converge


### Make the credible region

In [ ]:
# try and do sampling using uniform along diagonal
n_cr = 100
# new post
mu, sigma = get_example_posterior_param(q_obs[0:n_cr])

# posterior
post_cred = sps.multivariate_normal(mu,sigma)

# to get transformation to uniform along diagonal
eig_val, eig_vec = np.linalg.eig(sigma)

# sample for integral computation
width_dev = 7 # +/- 7 deviations from the mean
box_points = sps.uniform.rvs(-width_dev,2*width_dev,size=(100000,2))

# transform box points
transform_matrix = np.dot(eig_vec,np.sqrt(eig_val)*np.eye(2))
transform_box_points = np.dot(transform_matrix,box_points.T).T+mu

# area of box points
area_box = np.linalg.det(transform_matrix)*(2*width_dev)**2

# eval
pdf_out = post_cred.pdf(transform_box_points)

# make sure sampled area is close to 1
print(area_box*np.average(pdf_out))

In [ ]:
# visualization of how I am calculating the area here
plt.scatter(transform_box_points[:,0],transform_box_points[:,1])
test = post_cred.rvs(1000)
plt.scatter(test[:,0],test[:,1])
plt.plot(mu[0],mu[1],'ro')

In [ ]:
# get pdf in order of highest likelihood
ordered_pdf_out = np.flip(np.sort(pdf_out))

# sum terms until we get to probability 95%
areas = area_box*np.cumsum(ordered_pdf_out)/pdf_out.shape[0]

# get contour value where area reaches 95%
C_level = 0.95
ind = np.argwhere(areas>C_level)[0]
contour_t = ordered_pdf_out[ind]
print(contour_t)

**Credible Region**

The following region shows the credible region such that the area is 95%.

In [ ]:
# plot the 95% credible region
fig_credible_region, ax = plt.subplots(1)
fig_credible_region.set_figwidth(5)
ellipse_lam_dist_95 = make_cov_ellipse(mu,lam_cov,area=0.95,
                                       ellipse_args={'color':'r',
                                                     'alpha':0.2,'zorder':-1,
                                                     'label':'Wobbly-plate Distr.'})
ax.add_patch(ellipse_lam_dist_95)
region = transform_box_points[pdf_out>contour_t]
ax.scatter(region[:,0],region[:,1],label='Bayesian Posterior CR')
ax.set_title('{}% credible region $n={}$'.format(int(100*C_level),n_cr))
ax.set_xlim(-1,3)
ax.set_ylim(-1,3)
ax.set_aspect('equal')

ax.legend()

# save fig to master
this_fig_name = 'bayes_param_credible.png'
fig_all_master[this_fig_name] = fig_credible_region



# Bayesian Solution to Parameter Distribution Problem


(consider page 120 of Kaipio)

In the parameter distribution problem, we assume that the plate is wobbling. In this case, it is not errors in our measurement instruments, or additive noise, that is causing the variation in $q$, but variation in sampled values of the slope $\lambda$.

Suppose that we think the distribution of $\lambda$ is normally distributed with an unknown mean $\mu$ and unknown variance $\Sigma$. Our goal is to find the best estimate of the mean and variance and quantify the uncertainty in these parametric estimates of the distribution parameters.

We can determine the likelihood function by using a transformation of variables. Given $(\mu,\Sigma)$, we know that the distribution of $q$ will be:

\begin{align*}
q \mid \mu,\Sigma = \text{mv_normal}(X\mu+y_0,X\Sigma X^T)
\end{align*}

We choose a hyperprior for $(\mu,\Sigma)$ so that the distribution of $q$ will be normal-inverse-wishart. This will allow us to compute exact solutions for the Bayesian posterior distribution.

Our Bayesian model will be:

\begin{align*}
\vec{\lambda} &\sim \mathcal{NIW}(\mu_0,\kappa_0,\nu,\Psi_0) \\
Q(\vec{\lambda}) &= X\vec{\lambda} + y_0 \quad,\quad \text{where } X=\begin{pmatrix}0.6 & 0.7 \\
0.8 & 0.6
\end{pmatrix} \\
q \mid \mu,\Sigma &\sim \text{mv_normal}(\mu_q,\Sigma_q)
\end{align*}

where $\mu_q=X\mu +y_0$ and $\Sigma_q=X\Sigma X^T$.

Consider the distributions of $\mu_q$ and $\Sigma_q$:

\begin{align}
\Sigma_q = X\Sigma X^T &\sim \mathcal{IW}(\nu,X\Psi_0 X^T) \\
\mu_q \mid \Sigma_q &\sim N(X\mu_0+y_0,\kappa_0^{-1}\Sigma_q)
\end{align}

This implies that the joint distribution of $(\mu_q,\Sigma_q)$ will be normal-inverse-wishart.

From here, we can compute the posterior parameters of $(\mu_q,\Sigma_q)$ using the standard conjugacy applications.

Then we can obtain the hyper-parameters of interest by applying the inverse transformation of variables.

In [ ]:
plt.scatter(q_obs.T[0],q_obs.T[1])

In [ ]:
# hyper-parameters for prior

# covariance prior: 
# we want choose values so that the MAP of the prior is centered around a spherical covariance
# recall that the mode of IW is Psi/(nu+p+1)
nu = 5 # how much weight to put towards prior
Psi_prior = (nu+2+1)*prior_cov # use prior covariance defined earlier

# define prior covariance distribution
cov_prior_dist = sps.invwishart(nu,Psi_prior)
print('Mode of Cov Prior: ', cov_prior_dist.mode())
print()

# mean prior:
# we choose the mean to be 0 (plate is flat)
# we want to choose kappa so that the mean is apriori within 2 standard deviations
# from the mean
mean_prior = np.zeros(2)
kappa_prior = 4
mean_prior_dist = sps.multivariate_normal(mean_prior,1/kappa_prior*cov_prior_dist.mode())
print('Mode (mean) of Prior Mean: ', mean_prior_dist.mean)


In [ ]:
1/q_obs.shape[0]*np.dot((q_obs-q_obs.mean(axis=0)).T,q_obs-q_obs.mean(axis=0))

In [ ]:
np.outer(q_obs.mean(axis=0),q_obs.mean(axis=0))

In [ ]:
# hyper-parameters for posterior
this_info = {'mu0': mean_prior,
             'kappa0': kappa_prior,
             'nu0': nu,
             'Psi0': Psi_prior,
             'linear_map': locations,
             'y0': y0
            }

def get_hyper_post_params(sample, info=this_info):
    # get prior params
    mu0 = info['mu0']
    k0 = info['kappa0']
    nu0 = info['nu0']
    Psi0 = info['Psi0']
    
    # get map info
    X = info['linear_map']
    X_inv = np.linalg.inv(X)
    y0 = info['y0']
    
    # get data params
    n = sample.shape[0]
    qbar = sample.mean(axis=0)
    SSq = n*np.cov(sample.T,ddof=0)
    
    # get lam mean from data
    lam_bar = np.dot(X_inv,qbar-y0)
    
    # mu post
    mu_post = (k0*mu0+n*lam_bar)/(k0+n)
    
    # kappa and nu post
    k_post = k0+n
    nu_post = nu0+n
    
    # psi post 
    # psi: data term
    SS_lam = np.linalg.multi_dot([X_inv,SSq,X_inv.T])
    
    # psi: bias term
    bias_term = k0*n/(k0+n)*np.outer(lam_bar-mu0,lam_bar-mu0)
    
    # combined
    psi_post = Psi0 + SS_lam + bias_term 
    
    return mu_post, k_post, nu_post, psi_post


The following figure shows the hierarchical Bayesian model converging to the appropriate target distribution for the Wobbly-plate (as opposed to a point estimate).

In [ ]:
fig_dist_converge, axes = plt.subplots(2,2)
fig_dist_converge.set_figheight(8)
fig_dist_converge.set_figwidth(8)
fig_dist_converge.tight_layout(h_pad=2)

axes = axes.flatten()
n_size = [0,10,100,250,q_obs.shape[0]]
for i, ax in enumerate(axes):
#     ax.plot(lam_mean[0],lam_mean[1],'or',label='Fixed Slope')
    if i == 0:
        this_prior_dist = sps.multivariate_normal(mean_prior_dist.mean,cov_prior_dist.mode())
        Z_prior = this_prior_dist.pdf(eval_lamXY.T).reshape(lamX.shape)
        ax.contour(lamX,lamY,Z_prior)
        ax.set_title('Prior')
    else:
        (mu_p,k_p,nu_p,psi_p) = get_hyper_post_params(q_obs[0:n_size[i]])
        sigma_MAP = psi_p/(nu_p+2+1)
        mu_MAP = mu_p
        this_post_pdf = sps.multivariate_normal(mu_MAP,sigma_MAP)
        Z_post = this_post_pdf.pdf(eval_lamXY.T).reshape(lamX.shape)
        ax.contour(lamX,lamY,Z_post)
        ax.set_title('Distribution of $\lambda$, $n={}$'.format(n_size[i]))

# figure name will be 
this_fig_name = 'bayes_dist_converge.png'
fig_all_master[this_fig_name] = fig_dist_converge

# save only this fig
# fig_dist_converge.savefig('../'+this_fig_name)

The following figure shows how the hyper-parameters of the Bayesian hierarchical model are converging (to the appropriate point-estimates of the hyper-parameters).

In [ ]:
fig_hyper_converge, axes = plt.subplots(2,3)
fig_hyper_converge.set_figheight(6)
fig_hyper_converge.set_figwidth(8)
fig_hyper_converge.tight_layout(h_pad=1.5)

n_size = [0,50,250]
for j,n in enumerate(n_size):   
    # plot the covariance samples
    if j==0:
        this_cov_sample = cov_prior_dist.rvs(1000)
        this_mu_sample = np.array([sps.multivariate_normal.rvs(mean_prior,1/kappa_prior*cov) for cov in this_cov_sample])
    else:
        (mu_p,k_p,nu_p,psi_p) = get_hyper_post_params(q_obs[0:n])
        this_cov_sample = sps.invwishart.rvs(nu_p,psi_p,size=1000)
        this_mu_sample = np.array([sps.multivariate_normal.rvs(mu_p,1/k_p*cov) for cov in this_cov_sample])
    
    # plot the covariances
    cov_by_components = np.moveaxis(this_cov_sample,0,-1).reshape((4,-1))
    cov_low = np.quantile(cov_by_components,0.01)
    cov_high = np.quantile(cov_by_components,0.99)
    x = np.linspace(cov_low,cov_high,500)
    
    label_list = ['$s_{11}$','$s_{12}$','$s_{21}$','$s_{22}$']
    for k,comp in enumerate(cov_by_components):
        if k==2:
            # component 1 and 2 are the same
            # because covariance is symmetric
            continue       
        kde = sps.gaussian_kde(comp)
        axes[0,j].plot(x,kde(x),label=label_list[k])
        
        if j==0:
            axes[0,j].set_title('Prior')
        else:
            axes[0,j].set_title('Post $n={}$'.format(n))
        axes[0,j].legend(handlelength=0.9)
    
    
    # plot the means
    means_by_components = this_mu_sample.T
    mu_low = np.quantile(means_by_components,0.01)
    mu_high = np.quantile(means_by_components,0.99)
    y = np.linspace(mu_low,mu_high,500)
    
    for k,comp in enumerate(means_by_components):
        kde = sps.gaussian_kde(comp)
        axes[1,j].plot(y,kde(y),label='$\mu_{{ {} }}$'.format(k+1))
        
        if j==0:
            axes[1,j].set_title('Prior')
        else:
            axes[1,j].set_title('Post $n={}$'.format(n))
    axes[1,j].legend(handlelength=0.9)
    

# annotate axes change
for arN in np.arange(2):
    adj = -0.05 if arN==0 else 0.15
    bbox_props = dict(boxstyle="rarrow", fc=(0.8, 0.9, 0.9), ec="b", lw=2)
    t = axes[arN,0].text(1+adj, -0.05, "     ", ha="center", va="top", rotation=180,
                size=10,transform=axes[arN,0].transAxes,
                bbox=bbox_props)
    bb = t.get_bbox_patch()
    bb.set_boxstyle("rarrow", pad=0.15)


# axes[0,0].annotate('DETAIL',xy=(150,0.),xycoords='axes points')
    
# figure name will be 
this_fig_name = 'bayes_hyper_converge.png'
fig_all_master[this_fig_name] = fig_hyper_converge

# Save only this fig
# fig_hyper_converge.savefig('../'+this_fig_name)

This is the same figure as previous, but I re-scale the axes for the presentation.

In [ ]:
fig_hyper_present_converge, axes = plt.subplots(2,3)
fig_hyper_present_converge.set_figheight(6)
fig_hyper_present_converge.set_figwidth(8)
fig_hyper_present_converge.tight_layout(h_pad=1.5)

n_size = [0,50,250]

# start by generating the samples
saved_samples = {'cov': [], 'mean': []}
for j,n in enumerate(n_size):   
    # plot the covariance samples
    if j==0:
        this_cov_sample = cov_prior_dist.rvs(1000)
        this_mu_sample = np.array([sps.multivariate_normal.rvs(mean_prior,1/kappa_prior*cov) for cov in this_cov_sample])
    else:
        (mu_p,k_p,nu_p,psi_p) = get_hyper_post_params(q_obs[0:n])
        this_cov_sample = sps.invwishart.rvs(nu_p,psi_p,size=1000)
        this_mu_sample = np.array([sps.multivariate_normal.rvs(mu_p,1/k_p*cov) for cov in this_cov_sample])
    
    # save the samples
    saved_samples['cov'].append(this_cov_sample)
    saved_samples['mean'].append(this_mu_sample)

# get a common lower bound for all covariance plotting
cov_all_post = np.moveaxis(np.array(saved_samples['cov'])[1::],[0,1],[-1,-2]).reshape(-1,)
cov_low = np.quantile(cov_all_post,0.01)*1.5
cov_high = np.quantile(cov_all_post,0.99)*1.65
x = np.linspace(cov_low,cov_high,500)

mean_all_post = np.array(saved_samples['mean'])[1::].reshape(-1,)
mu_low = np.quantile(mean_all_post,0.01)*1.5
mu_high = np.quantile(mean_all_post,0.99)*1.5
y = np.linspace(mu_low,mu_high,500)

for j,n in enumerate(n_size):   
    # load the samples
    this_cov_sample = saved_samples['cov'][j]
    this_mu_sample = saved_samples['mean'][j]
    
    # plot the covariances
    cov_by_components = np.moveaxis(this_cov_sample,0,-1).reshape((4,-1))
    
    
    label_list = ['$s_{11}$','$s_{12}$','$s_{21}$','$s_{22}$']
    for k,comp in enumerate(cov_by_components):
        if k==2:
            # component 1 and 2 are the same
            # because covariance is symmetric
            continue       
        kde = sps.gaussian_kde(comp)
        axes[0,j].plot(x,kde(x),label=label_list[k])
        
        if j==0:
            axes[0,j].set_title('Prior')
        else:
            axes[0,j].set_title('Post $n={}$'.format(n))
        axes[0,j].legend(handlelength=0.9)
    
    
    # plot the means
    means_by_components = this_mu_sample.T
    
    for k,comp in enumerate(means_by_components):
        kde = sps.gaussian_kde(comp)
        axes[1,j].plot(y,kde(y),label='$\mu_{{ {} }}$'.format(k+1))
        
        if j==0:
            axes[1,j].set_title('Prior')
        else:
            axes[1,j].set_title('Post $n={}$'.format(n))
    axes[1,j].legend(handlelength=0.9)
    
# change y-scale of axes
lower_y = np.array(axes[0,1].get_ylim())/4
axes[0,0].set_ylim(lower_y)

lower_y = np.array(axes[0,1].get_ylim())/1
axes[1,0].set_ylim(lower_y)

    
    
# # annotate axes change
# for arN in np.arange(2):
#     adj = -0.05 if arN==0 else 0.15
#     bbox_props = dict(boxstyle="rarrow", fc=(0.8, 0.9, 0.9), ec="b", lw=2)
#     t = axes[arN,0].text(1+adj, -0.05, "     ", ha="center", va="top", rotation=180,
#                 size=10,transform=axes[arN,0].transAxes,
#                 bbox=bbox_props)
#     bb = t.get_bbox_patch()
#     bb.set_boxstyle("rarrow", pad=0.15)


# axes[0,0].annotate('DETAIL',xy=(150,0.),xycoords='axes points')
    
# figure name will be 
this_fig_name = 'bayes_hyper_present_converge.png'
fig_all_master[this_fig_name] = fig_hyper_present_converge

# Save only this fig
# fig_hyper_present_converge.savefig('../'+this_fig_name,
#                                    dpi=250,bbox_inches='tight')

In [ ]:
axes[1,1].get_ylim()

In [ ]:
(np.array(saved_samples['cov'])[0:4]).shape

In [ ]:
np.array(saved_samples['cov'])[1::,1,:,:]

In [ ]:
this_cov_test[:,:,1,2]

In [ ]:
this_cov_test = np.moveaxis(np.array(saved_samples['cov'])[1::],[0,1],[-1,-2]).reshape(-1,)
print(this_cov_test.shape)
np.quantile(this_cov_test,0.01)

In [ ]:
for item in current_axes_lim.keys():
    print(item, current_axes_lim[item])
    print()
    
cov_lower_lims = [current_axes_lim['cov'][j][0] for j in [1,2]]
print(cov_lower_lims)

# SAVE ALL FIGS

In [ ]:
# make sure all the figures are there
for key in fig_all_master.keys():
    print(key)

In [ ]:
# check individual figures
check_name = None
if check_name == None:
    print()
else:
    fig_all_master[check_name]

In [ ]:
# # # save all figs
# for figfilename in fig_all_master:
#     fig_all_master[figfilename].savefig('../'+figfilename,
#                                         dpi=250,bbox_inches='tight')